## Demo

In [ ]:
%pip install langchain 
%pip install langchain_chroma 
%pip install langchain_community 
%pip install langchain_core
%pip install langchain_google_vertexai
%pip install langchain_text_splitters
%pip install jq
# %pip install bs4

In [3]:
# import bs4
# from langchain import hub
# from langchain_chroma import Chroma
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_google_vertexai import VertexAIEmbeddings

# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# # Retrieve and generate using the relevant snippets of the blog.
# retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")


# def format_docs(docs):
#     return "\n\n".join(doc for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")

In [63]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_chroma import Chroma
from google.oauth2.service_account import Credentials

credentials = Credentials.from_service_account_file(
    "ive-fyp-436703-3a208c6d96a0.json")

vertexAIEmbeddings = VertexAIEmbeddings(
        credentials=credentials,
        project=credentials.project_id,
        model_name="text-multilingual-embedding-002",
    )

vectorstore = Chroma(persist_directory='./chroma_db', embedding_function=vertexAIEmbeddings)

In [64]:
from langchain import hub

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
prompt = hub.pull("rlm/rag-prompt")

C:\Users\karlc\AppData\Roaming\Python\Python312\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [69]:
from langchain_google_vertexai import ChatVertexAI
from google.oauth2.service_account import Credentials

credentials=Credentials.from_service_account_file("ive-fyp-436703-3a208c6d96a0.json")

llm = ChatVertexAI(
    model="gemini-1.0-pro",
    temperature=0.1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    credentials=credentials,
    project=credentials.project_id,
    task="text-generation",
    region="us-central1",
)

In [70]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What routes from KMB are avalable at AMOY GARDENS station?")

"I'm sorry, but the provided context does not contain information about the routes available at AMOY GARDENS station. Therefore, I cannot answer your question. \n"

In [71]:
retriever.invoke("AMOY GARDENS")

[]

In [46]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])